In [1]:
def compute_longitudinal_safe_distance(speed, reaction_time=0.001, friction=0.5, gravity=9.81, vehicle_length = 4.8):
    """
    计算纵向安全包络（前后安全距离）
    
    参数:
        speed (float): 车辆当前速度 (m/s)
        reaction_time (float): 反应时间 (s)
        friction (float): 路面摩擦系数
        gravity (float): 重力加速度 (m/s^2)
        
    返回:
        safe_distance (float): 纵向安全包络距离 (m)
    """
    braking_distance = (speed ** 2) / (2 * friction * gravity)
    reaction_distance = speed * reaction_time
    safe_distance = braking_distance + reaction_distance + vehicle_length
    return safe_distance

# 计算横向安全包络
def compute_lateral_safe_distance(speed, vehicle_width=2.16, turn_radius=3.68):
    """
    计算横向安全包络（侧向安全距离）
    
    参数:
        vehicle_width (float): 车辆宽度 (m)
        speed (float): 车辆当前速度 (m/s)
        turn_radius (float): 车辆的最小转弯半径 (m)
        
    返回:
        lateral_safe_distance (float): 横向安全包络距离 (m)
    """
    lateral_offset = (speed ** 2) / (2 * turn_radius)  # 侧向偏移量
    return vehicle_width + 2 * lateral_offset

In [2]:
import os
import json

def load_json_file(file_path):
    """读取 JSON 文件并返回数据"""
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def find_key_frames(record_dir):
    """遍历指定文件夹，找到满足安全距离条件的关键帧"""
    Key_frame = []

    # 获取所有 JSON 文件，并按文件名排序
    json_files = sorted([f for f in os.listdir(record_dir) if f.endswith(".json")])

    # 逐个读取并解析 JSON 文件
    for json_file in json_files:
        file_path = os.path.join(record_dir, json_file)
        data = load_json_file(file_path)

        # 提取 NPC 和 ego 数据
        npc1_lateral_distance = data["npc1"]["lateral_distance"]
        npc1_longitudinal_distance = data["npc1"]["longitudinal_distance"]
        npc2_lateral_distance = data["npc2"]["lateral_distance"]
        npc2_longitudinal_distance = data["npc2"]["longitudinal_distance"]
        ego_speed = data["ego_vehicle"]["speed"]

        # 计算纵向和横向安全距离
        longitudinal_safe_distance = compute_longitudinal_safe_distance(ego_speed)
        lateral_safe_distance = compute_lateral_safe_distance(ego_speed)

        # 判断是否满足安全距离条件
        if (longitudinal_safe_distance > npc1_longitudinal_distance or 
            longitudinal_safe_distance > npc2_longitudinal_distance or
            lateral_safe_distance > npc1_lateral_distance or 
            lateral_safe_distance > npc2_lateral_distance):
            Key_frame.append(json_file)

    return Key_frame

In [3]:
record_dir = "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/record"
key_frames = find_key_frames(record_dir)

# 输出结果
print(key_frames)
print(len(key_frames))

['0000.json', '0001.json', '0002.json', '0003.json', '0004.json', '0005.json', '0006.json', '0007.json', '0008.json', '0009.json', '0010.json', '0011.json', '0012.json', '0013.json', '0014.json', '0015.json', '0016.json', '0017.json', '0018.json', '0019.json', '0020.json', '0021.json', '0022.json', '0023.json', '0024.json', '0025.json', '0026.json', '0027.json', '0028.json', '0029.json', '0030.json', '0031.json', '0032.json', '0033.json', '0034.json', '0035.json', '0036.json', '0037.json', '0038.json', '0039.json', '0040.json', '0041.json', '0042.json', '0043.json', '0044.json', '0045.json', '0046.json', '0047.json', '0048.json', '0049.json', '0050.json', '0051.json', '0052.json', '0053.json', '0054.json', '0055.json', '0056.json', '0057.json', '0058.json', '0059.json', '0060.json', '0061.json', '0062.json', '0063.json', '0064.json', '0065.json', '0066.json', '0067.json', '0068.json', '0069.json', '0070.json', '0071.json', '0072.json', '0073.json', '0074.json', '0075.json', '0076.json'

In [4]:
import os

def track_action_changes(record_dir, key_frame):
    action_sequence_npc1 = []
    last_action_npc1 = None
    action_sequence_npc2 = []
    last_action_npc2 = None
    action_change_frame = set()

    for json_file in key_frame:
        file_path = os.path.join(record_dir, json_file)
        data = load_json_file(file_path)
        current_npc1_action = data["npc1"]["action"]
        if current_npc1_action != last_action_npc1:
            action_sequence_npc1.append(current_npc1_action)
            last_action_npc1 = current_npc1_action
            action_change_frame.add(file_path)

        current_npc2_action = data["npc2"]["action"]
        if current_npc2_action != last_action_npc2:
            action_sequence_npc2.append(current_npc2_action)
            last_action_npc2 = current_npc2_action
            action_change_frame.add(file_path)


    action_change_frame = sorted(action_change_frame)
    return action_sequence_npc1, action_sequence_npc2, action_change_frame



In [5]:
# 示例调用
record_dir = "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/record"
action_sequence_npc1, action_sequence_npc2, action_change_frames = track_action_changes(record_dir, key_frames)

# 输出结果
print("NPC 1 Action 变化序列:", action_sequence_npc1)
print("NPC 2 Action 变化序列:", action_sequence_npc2)
print("所有 action 变化发生的文件:", action_change_frames)
print("动作变化文件数量:", len(action_change_frames))

NPC 1 Action 变化序列: [1, 0, 3, 2, 0, 1, 0, 2]
NPC 2 Action 变化序列: [2, 0, 1, 3, 2, 3, 2, 1, 2, 0, 1, 2]
所有 action 变化发生的文件: ['/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/record/0000.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/record/0046.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/record/0074.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/record/0112.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/record/01

In [2]:
import json

def format_float(value):
    return f"{value:.2f}"

def generate_npc_prompt(npc_key, npc_data, ego_data):
    action_mapping = {
        0: "左变道",
        1: "右变道",
        2: "直行",
        3: "刹车"
    }

    action_text = action_mapping.get(npc_data["action"], "未知动作")
    relative_side = "右边" if npc_data["lateral_distance"] > 0 else "左边"
    front_back = "前方" if npc_data["longitudinal_distance"] > 0 else "后方"
    if npc_data['action'] == 0:  # 左变道
        target_lane = int(npc_data['start_lane']) + 1
    elif npc_data['action'] == 1:  # 右变道
        target_lane = int(npc_data['start_lane']) - 1
    else:
        target_lane = int(npc_data['start_lane'])
        
    if npc_data['current_lane'] == ego_data['lane']:
        prompt = f"{npc_key}和ego位于同一条车道, 在ego的{front_back}{format_float(npc_data['longitudinal_distance'])} m, 正准备以速度{format_float(npc_data['speed'])} m/s做{action_text},目标车道是{target_lane},"
        prompt += f"目标速度{format_float(npc_data['target_speed'])} m/s,距离当前车道中心{format_float(npc_data['lateral_distance'])} m。"
    else:
        prompt = f"{npc_key}位于ego的{relative_side} {(abs(npc_data['current_lane'] - ego_data['lane']))}车道, 距离ego横向距离为{format_float(npc_data['lateral_distance'])} m, 在ego的{front_back}{format_float(npc_data['longitudinal_distance'])} m,"
        prompt += f"正在以速度速度{format_float(npc_data['speed'])} m/s做{action_text},目标车道是{target_lane},目标速度{format_float(npc_data['target_speed'])} m/s。"
    
    # 如果是直行或者刹车，则不需要转向角
    if npc_data["action"] in [2, 3]:
        prompt = prompt.replace(f"正在以速度速度{format_float(npc_data['speed'])} m/s做{action_text},目标车道是{target_lane}",f"正在以速度速度{format_float(npc_data['speed'])} m/s做{action_text}")
    
    return prompt

def generate_ego_prompt(ego_data):
    return f"ego位于车道{ego_data['lane']}, 当前速度{format_float(ego_data['speed'])} m/s, 当前加速度{format_float(ego_data['acceleration'])} m/s², 正在直行。"


def generate_full_prompt(json_file):
    """生成完整的场景描述prompt"""
    # 读取 JSON 文件
    data = load_json_file(json_file)

    prompts = []
    prompt = generate_ego_prompt(data["ego_vehicle"])
    prompts.append(prompt)
    # 遍历 NPC 并生成 prompt
    for npc_key, npc_data in data.items():
        if npc_key != "ego_vehicle":
            prompt = generate_npc_prompt(npc_key, npc_data, data["ego_vehicle"])
            prompts.append(prompt)

    # 合并开始和结束的描述
    # prompt_start = "这是场景的俯视图，在图片中，车道从左到右，分别为-3到-7,车道宽度为3.5m,ego车位于图片中间位置。"
    # prompt_end = ("请结合图片和描述判断,预测行为结束前30m内是否可能发生碰撞")
    # final_prompt = f"{prompt_start} " + " ".join(prompts) + f" {prompt_end}"
    prompt_start = "车道从左到右，分别为-3到-7,车道宽度为3.5m。"
    prompt_end = ("请一步一步分析。根据描述预测所有npc行为结束前30m内，是否可能发生碰撞。")
    final_prompt = f"{prompt_start} "+ " ".join(prompts) + f" {prompt_end}"

    return final_prompt



In [4]:
import json
import math

def load_json_file(file_path):
    """读取 JSON 文件并返回数据"""
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data
def format_float(value, precision=2):
    return round(float(value), precision) 

def calculate_incentive(npc_data):
    lateral_distance = abs(float(npc_data['lateral_distance']))  # 横向距离
    longitudinal_distance = float(npc_data['longitudinal_distance'])  # 纵向距离
    
    total_distance = math.sqrt(lateral_distance**2 + longitudinal_distance**2)
    return total_distance

# def generate_npc_prompt(npc_key, npc_data, ego_data):

#     relative_side = "右边" if npc_data["lateral_distance"] > 0 else "左边"
#     front_back = "前方" if npc_data["longitudinal_distance"] > 0 else "后方"
#     if npc_data['action'] == 0:  # 左变道
#         target_lane = int(npc_data['start_lane']) + 1
#     elif npc_data['action'] == 1:  # 右变道
#         target_lane = int(npc_data['start_lane']) - 1
#     else:
#         target_lane = int(npc_data['start_lane'])
        
#     if npc_data['current_lane'] == ego_data['lane']:
#         prompt = f"{npc_key}和ego位于同一条车道, 在ego的{front_back}{format_float(npc_data['longitudinal_distance']-4.8)} m, 以速度{format_float(npc_data['speed'])} m/s行驶"
#     else:
#         prompt = f"{npc_key}位于ego的{relative_side} {(abs(npc_data['current_lane'] - ego_data['lane']))}车道, 距离ego横向距离为{format_float(npc_data['lateral_distance'] - 2.16)} m, 在ego的{front_back}{format_float(npc_data['longitudinal_distance']-4.8)} m,"
#         prompt += f"以速度{format_float(npc_data['speed'])} m/s行驶。"
#     return prompt

# def generate_ego_prompt(ego_data):
#     return f"ego位于车道{ego_data['lane']}, 当前速度{format_float(ego_data['speed'])} m/s, 当前加速度{format_float(ego_data['acceleration'])} m/s², 正在直行。"



def generate_npc_prompt(npc_key, npc_data, ego_data):
    # Determine relative position (left or right)
    relative_side = "right" if npc_data["lateral_distance"] < 0 else "left"
    # # Determine if the NPC is in front or behind the ego vehicle
    # front_back = "front" if npc_data["longitudinal_distance"] > 0 else "behind"
    # 计算纵向关系
    if npc_data["longitudinal_distance"] < -4.8:  # NPC 完全在 ego 车前方
        front_back = "in front of ego"
        longitudinal_distance = abs(npc_data["longitudinal_distance"]) - 4.8
        distance_description = f"with NPC's rear {format_float(longitudinal_distance)} m in front of Ego's front"
    elif npc_data["longitudinal_distance"] > 4.8:  # NPC 完全在 ego 车后方
        front_back = "behind ego"
        longitudinal_distance = abs(npc_data["longitudinal_distance"]) - 4.8
        distance_description = f"with NPC's front {format_float(longitudinal_distance)} m beind Ego's rear"
    elif npc_data["longitudinal_distance"] < -0.2 and npc_data["longitudinal_distance"] > -4.8:  # NPC 部分重叠 ego 车,且稍领先ego
        front_back = "partially beside and slightly in front of ego"
        longitudinal_distance = abs(npc_data["longitudinal_distance"])  # 取最近的端点距离
        distance_description = f"with NPC's front {format_float(longitudinal_distance)} m in front of Ego's front"
    elif npc_data["longitudinal_distance"] > 0.2 and npc_data["longitudinal_distance"] < 4.8:
        front_back = "partially beside and slightly behind ego"
        longitudinal_distance = abs(npc_data["longitudinal_distance"])  # 取最近的端点距离
        distance_description = f"with NPC's front {format_float(longitudinal_distance)} m behind Ego's front"
    else:
        front_back = "nearly parallel with ego"
        distance_description = ""

    if npc_data["lateral_distance"] < -2.16: # NPC 完全在 ego 右边
        left_right = "to the right of ego"
        lateral_distance = abs(npc_data["lateral_distance"]) - 2.16  # NPC 车左端到 Ego 车右端的距离
        lateral_description = f"with NPC's left {format_float(lateral_distance)} m to the right of Ego's right"
    elif npc_data["lateral_distance"] > 2.16: # NPC 完全在 ego 左边
        left_right = "to the left of ego"
        lateral_distance = abs(npc_data["lateral_distance"]) - 2.16  # NPC 车右端到 Ego 车左端的距离
        lateral_description = f"with NPC's right {format_float(lateral_distance)} m to the left of Ego's left"
    # elif npc_data["lateral_distance"] > -2.16 and npc_data["lateral_distance"] < 0: # NPC 部分重叠 ego 车,且稍微在 ego 右边,
    elif npc_data["lateral_distance"] > -2.16 and npc_data["lateral_distance"] < -0.2: # NPC 部分重叠 ego 车,且稍微在 ego 右边,
        left_right = "partially beside and slightly to the right of ego"
        lateral_distance = abs(npc_data["lateral_distance"])  # NPC 车右端到 Ego 车左端的距离
        lateral_description = f"with NPC's right {format_float(lateral_distance)} m to the right of Ego's side"
    elif npc_data["lateral_distance"] < 2.16 and npc_data["lateral_distance"] > 0.2:
        left_right = "partially beside and slightly to the left of ego"
        lateral_distance = abs(npc_data["lateral_distance"])  # NPC 车左端到 Ego 车右端的距离
        lateral_description = f"with NPC's right {format_float(lateral_distance)} m to the right of Ego's side"
    else:
        left_right = "nearly overlapping laterally with ego"
        lateral_description = ""

    if npc_data['current_lane'] == ego_data['lane']:
        prompt = f"{npc_key} and ego are in the same lane."
        prompt += f"Laterally (horizontally), {npc_key} is {left_right},{lateral_description}, "
        prompt += f"and and longitudinally (vertically) {front_back}, {distance_description}, "
        prompt += f"traveling at a speed of {format_float(npc_data['speed'])} m/s."
    else:
        prompt = f"{npc_key} is in the {relative_side} {abs(npc_data['current_lane'] - ego_data['lane'])} lane. "
        prompt += f"Laterally (horizontally), {npc_key} is {left_right},{lateral_description}, "
        prompt += f"and and longitudinally (vertically) {front_back}, {distance_description}, "
        prompt += f"traveling at a speed of {format_float(npc_data['speed'])} m/s."

    return prompt


def generate_npc_prompt_easy(npc_key, npc_data, ego_data):
    # Determine relative position (left or right)
    relative_side = "right" if npc_data["lateral_distance"] < 0 else "left"
    # Determine if the NPC is in front or behind the ego vehicle
    front_back = "front" if npc_data["longitudinal_distance"] < 0 else "behind"

    if npc_data['current_lane'] == ego_data['lane']:
        prompt = (
            f"{npc_key} and ego are in the same lane, "
            f"{npc_key} is partially in the {relative_side} of ego, with {format_float(abs(npc_data['lateral_distance']))} m of lateral distance to the ego, and "
            f"is {front_back} of ego, with{format_float(abs(npc_data['longitudinal_distance']))} m of longitudinal distance to the ego, traveling at a speed of {format_float(npc_data['speed'])} m/s."
        )
    else:
        prompt = (
            f"{npc_key} is in the {relative_side} {abs(npc_data['current_lane'] - ego_data['lane'])} lane, "
            f"{npc_key} is in the {relative_side} of ego, with {format_float(abs(npc_data['lateral_distance']))} m lateral distance to the ego, and "
            f"is {front_back} of ego, with {format_float(abs(npc_data['longitudinal_distance']))} m longitudinal distance to the ego, traveling at a speed of {format_float(npc_data['speed'])} m/s."
        )
    return prompt

def generate_ego_prompt(ego_data):
    return f"ego is in lane {ego_data['lane']}, traveling at a speed of {format_float(ego_data['speed'])} m/s, with a current acceleration of {format_float(ego_data['acceleration'])} m/s², moving straight ahead."

def generate_full_prompt(json_file):
    """生成完整的场景描述prompt"""
    # 读取 JSON 文件
    data = load_json_file(json_file)

    prompts = []
    prompt = generate_ego_prompt(data["ego_vehicle"])
    prompts.append(prompt)
    # 遍历 NPC 并生成 prompt
    for npc_key, npc_data in data.items():
        if npc_key != "ego_vehicle":
            prompt = generate_npc_prompt(npc_key, npc_data, data["ego_vehicle"])
            prompts.append(prompt)

    # 合并开始和结束的描述
    final_prompt = f"In the BEV image, the lanes from left to right are numbered from -2, to -4, with a lane width of 3.5 meters. " + " ".join(prompts)
    # final_prompt_easy = f"In the BEV image, the lanes from left to right are numbered from -3 to -7, with a lane width of 3.5 meters. The ego vehicle is located in the center of the image." 
    # final_prompt_easy += f"Each vehicle has a width of 2.16 m and a length of 4.8 m, and the distances are measured from the each vehicles' center position." + " ".join(prompts)
    return final_prompt



In [8]:
import base64
from pydantic import BaseModel
from enum import Enum
from typing import List
from openai import OpenAI
import cv2
import matplotlib.pyplot as plt
class CorrectionAction(str, Enum):
    LEFT_LANE_CHANGE = "左变道"
    RIGHT_LANE_CHANGE = "右变道"
    ACCELERATE_STRAIGHT = "加速直行"
    DECELERATE_STRAIGHT = "减速直行"

# class probability(str, Enum):
#     LEFT_LANE_CHANGE = "左变道"
#     RIGHT_LANE_CHANGE = "右变道"
#     ACCELERATE_STRAIGHT = "加速直行"
#     DECELERATE_STRAIGHT = "减速直行"

class RepairAction(BaseModel):
    action: CorrectionAction
    speed: float  # 限定速度为浮动值

class CollisionPredict(BaseModel):
    collision_scores: int  
    reason: str
    predict_npc_action:str
    # most_likely_collision_vehicle: str  # 如果发生碰撞，提供最有可能发生碰撞的车辆名称
    # correction_action: List[RepairAction]

    def to_dict(self):
        return self.model_dump()  # 提供修正行动和相关速度

def get_collision_reasoning(final_prompt):
    
    client = OpenAI(
    base_url="https://yunwu.ai/v1",
    api_key='sk-Ko0JaD1xNhI3xJI7gCwUYdRTuCcoDnlJOUdZxS9vJxo33X9O',
    timeout=120)


    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are a helpful collision prediction and autonomous driving assistant."},
            {"role": "user", "content": final_prompt},
        ],
        temperature=0,
        response_format=CollisionPredict
    )
    collision_reasoning = completion.choices[0].message.parsed
    return collision_reasoning

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def crop_and_encode(image_path):
    """
    读取图像 -> 确保文件存在 -> 确保图像可读取 -> 中心裁剪 -> Base64 编码
    """

    if not os.path.exists(image_path):
        raise FileNotFoundError(f"文件不存在: {image_path}")
    
    RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT = 200, 250
    # 读取图像
    image = cv2.imread(image_path)

    if image is None:
        raise ValueError(f"无法读取图像: {image_path}")

    # 获取尺寸
    height, width, _ = image.shape

    # 计算裁剪范围
    crop_x_center, crop_y_center = width // 2, height // 2
    left, right = max(0, crop_x_center - RESIZED_IMAGE_WIDTH // 2), min(width, crop_x_center + RESIZED_IMAGE_WIDTH // 2)
    top, bottom = max(0, crop_y_center - RESIZED_IMAGE_HEIGHT // 2), min(height, crop_y_center + RESIZED_IMAGE_HEIGHT // 2)

    cropped_image = image[top:bottom, left:right]
    plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Cropped Image")
    plt.show()
    # Base64 编码
    _, buffer = cv2.imencode('.png', cropped_image)
    encoded_image = base64.b64encode(buffer).decode('utf-8')

    return encoded_image


def get_collision_reasoning_with_image(final_prompt, image_path):
    client = OpenAI(
        base_url="https://yunwu.ai/v1",
        api_key='sk-Ko0JaD1xNhI3xJI7gCwUYdRTuCcoDnlJOUdZxS9vJxo33X9O',
        timeout=120)
    
    img_b64_str = encode_image(image_path)
    # 发送包含图像和文字提示的请求
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "你是一个自动驾驶事故预测员，你的任务是根据历史数据做出0.5-1秒的预测，是否会发生事故，你需要考虑npc的行为，包括车道变换，加速和减速。并且按照以下步骤来分析"
            "1.视觉分析：基于bev图像，识别所有npc车辆，关注npc的车头朝向，速度变化。"
            "2.文本融合：结合文本信息，再次改进视觉分析结果"
            "3.根据融合后的信息，预测未来1秒内的碰撞风险。"},
            {"role": "user", 
             "content": [
                 {"type": "text","text":final_prompt},
                 {
                  "type":"image_url",
                  "image_url":{
                      "url":f"data:image/png;base64,{img_b64_str}"}
                  }
                 ]}
        ],
        temperature=0,
        # response_format=CollisionPredict
    )
    
    # 解析和返回推理结果
    collision_reasoning = completion.choices[0].message.parsed
    return collision_reasoning


In [32]:
get_collision_reasoning_with_image("图片中心位置的车辆是ego，其他的车辆为npc，请根据图片分析图片中的npc车辆的行为","/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/bev/0053.png")

CollisionPredict(final_answer=False, reason='The NPC vehicle on the right is positioned in its lane without any indication of turning or changing lanes. There are no visible obstacles or signals suggesting a change in behavior, so it is likely to continue moving straight.', predict_npc_action='The NPC vehicle on the right is likely to continue moving straight in its lane.')

In [12]:
crop_and_encode("/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_uniad_traj/collision/RouteScenario_1792_rep2_Town12_HazardAtSideLane_4_None_03_30_10_04_01/bev/0000.png")

'...'

In [14]:
def save_prompt_record(prompt_records, save_path="prompt_records.json"):
    # 读取现有的数据，如果存在的话
    if os.path.exists(save_path):
        with open(save_path, "r", encoding="utf-8") as f:
            existing_data = json.load(f)
    else:
        existing_data = []

    # 追加新的记录
    existing_data.extend(prompt_records)

    # 写回数据到文件
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)

    print(f"Prompt 记录已追加到 {save_path}")

def get_collision_reasoning_with_multi_image(client, final_prompt, record_paths):
    
    # 初始化 messages
    messages = [
        # {"role": "system", "content": "你是一个自动驾驶碰撞预测员，你的任务是根据历史数据做出0.5-1秒的预测，车辆行为预测，预测这些车辆是否会发生碰撞，你需要考虑车辆的的行为，包括车道变换，加速和减速。"},
            # "**1**.视觉分析：基于bev图像，识别所有npc车辆，关注npc的车头朝向，速度变化，每张图片间隔 0.5s。"
            # "**2**.文本融合：结合文本信息，再次改进视觉分析结果"
            # "**3**.根据融合后的信息，预测未来1秒内的碰撞风险。"},
        # {"role": "system", "content": "You are an autonomous driving collision predictor. Your task is to make predictions within 0.5-1 seconds based on historical data, "
        # "predicting whether collisions will occur. You need to predict vehicle behaviors, including lane changes, acceleration, and deceleration."},
        {"role": "system", "content":"""You are an **autonomous driving collision prediction expert**. Your task is to predict whether a collision will occur **within the next 0.5-1 second** based on **bev image data and scenario description text data**.",
        "Your role is to evaluate the provided scenario based on the following safety metrics with scores ranging from 0 to 5 for each metric, where 0 indicates collision and 5 indicates no risk of collision:"

            **Risk Levels and Definitions:**
            <Extreme Risk (Score: 1)>: Immediate collision or very high likelihood of impact. Urgent action is required.
            <High Risk (Score: 2)>: Close to collision or highly probable collision path. Needs prompt attention and quick manoeuvring.
            <Medium Risk (Score: 3)>: Moderate collision risk but manageable with timely reactions and proper strategies.
            <Low Risk (Score: 4)>: Minimal collision risk. The situation is controllable with sufficient time to react.
            <Negligible Risk (Score: 5)>: No significant collision risk. Obstacles are either moving away or far enough not to interfere." }"
                                    
            ###Output the reasoning for each step in the following format:
            step1:
            step2:
            step3:
            step4:
                                    """},
        {"role": "user", "content": []}
    ]
    
    prompt_records = []
    messages[1]["content"].append(
        {"type": "text", "text": final_prompt}
    )
    # 添加最终问题
    final_record = {
        "path": "Final Prompt",
        "prompt": final_prompt
    }
    prompt_records.append(final_record)
    
    # 遍历所有图片，分别添加描述
    for file_i in range(len(record_paths)):
        
        prompt_i = generate_full_prompt(record_paths[file_i])
            # 构造单条记录
        prompt_record = {
            "path": record_paths[file_i],  # 存储完整路径
            "prompt": prompt_i
        }

        prompt_records.append(prompt_record)  # 添加到列表
        # 获取碰撞推理结果
        json_file_name = os.path.basename(record_paths[file_i])  # 获取 '0000.json'
        base_name = os.path.splitext(json_file_name)[0]  # 去掉扩展名，得到 '0000'

        # 获取根目录路径
       root_dir = os.path.dirname(os.path.dirname(record_paths[file_i]))  # 获取到 "RouteScenario_1773_rep0_Town06_FreeRide_2_None_03_15_14_33_21" 目录

        # 构造 BEV 文件路径（与 JSON 文件名对应）
        bev_file_path = os.path.join(root_dir, "bev", f"{base_name}.png")

        img_b64_str = encode_image(bev_file_path)


        messages[1]["content"].append(
            {"type": "text", "text": f"This is the {file_i+1}th image, {prompt_i}."}
        )
        messages[1]["content"].append(
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_b64_str}"}}
        )
    print(prompt_records)
    save_prompt_record(prompt_records)
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-11-20",
        messages=messages,
        temperature=0.8,
        # response_format=CollisionPredict
    )
    
    # 解析和返回推理结果
  # collision_reasoning = completion.choices[0].message.parsed
  # return collision_reasoning.to_dict()
    collision_reasoning = completion.choices[0].message.content
    return collision_reasoning

In [9]:
def save_prompt_record_no_image(prompt_records, save_path="prompt_records_no_image.json"):
    # 读取现有的数据，如果存在的话
    if os.path.exists(save_path):
        with open(save_path, "r", encoding="utf-8") as f:
            existing_data = json.load(f)
    else:
        existing_data = []

    # 追加新的记录
    existing_data.extend(prompt_records)

    # 写回数据到文件
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)

    print(f"Prompt 记录已追加到 {save_path}")
def get_collision_reasoning_with_no_image(client, final_prompt, record_paths):
    
    # 初始化 messages
    messages = [
        # {"role": "system", "content": "你是一个自动驾驶碰撞预测员，你的任务是根据历史数据做出0.5-1秒的预测，车辆行为预测，预测这些车辆是否会发生碰撞，你需要考虑车辆的的行为，包括车道变换，加速和减速。"},
            # "**1**.视觉分析：基于bev图像，识别所有npc车辆，关注npc的车头朝向，速度变化，每张图片间隔 0.5s。"
            # "**2**.文本融合：结合文本信息，再次改进视觉分析结果"
            # "**3**.根据融合后的信息，预测未来1秒内的碰撞风险。"},
        # {"role": "system", "content": "You are an autonomous driving collision predictor. Your task is to make predictions within 0.5-1 seconds based on historical data, "
        # "predicting whether collisions will occur. You need to predict vehicle behaviors, including lane changes, acceleration, and deceleration."},
        {"role": "system", "content":"""You are an **autonomous driving collision prediction expert**. Your task is to predict whether a collision will occur **within the next 0.5-1 second** based on **bev image data and scenario description text data**.",
        "Your role is to evaluate the provided scenario based on the following safety metrics with scores ranging from 0 to 5 for each metric, where 0 indicates collision and 5 indicates no risk of collision:"

            **Risk Levels and Definitions:**
            <Extreme Risk (Score: 1)>: Immediate collision or very high likelihood of impact. Urgent action is required.
            <High Risk (Score: 2)>: Close to collision or highly probable collision path. Needs prompt attention and quick manoeuvring.
            <Medium Risk (Score: 3)>: Moderate collision risk but manageable with timely reactions and proper strategies.
            <Low Risk (Score: 4)>: Minimal collision risk. The situation is controllable with sufficient time to react.
            <Negligible Risk (Score: 5)>: No significant collision risk. Obstacles are either moving away or far enough not to interfere." }"
                                    """},
        {"role": "user", "content": []}
    ]
    
    prompt_records = []
    messages[1]["content"].append(
        {"type": "text", "text": final_prompt}
    )
    # 添加最终问题
    final_record = {
        "path": "Final Prompt",
        "prompt": final_prompt
    }
    prompt_records.append(final_record)
    
    # 遍历所有图片，分别添加描述
    for file_i in range(len(record_paths)):
        
        prompt_i = generate_full_prompt(record_paths[file_i])
            # 构造单条记录
        prompt_record = {
            "path": record_paths[file_i],  # 存储完整路径
            "prompt": prompt_i
        }

        prompt_records.append(prompt_record)  # 添加到列表
        # 获取碰撞推理结果
        json_file_name = os.path.basename(record_paths[file_i])  # 获取 '0000.json'
        base_name = os.path.splitext(json_file_name)[0]  # 去掉扩展名，得到 '0000'

        # 获取根目录路径
       root_dir = os.path.dirname(os.path.dirname(record_paths[file_i]))  # 获取到 "RouteScenario_1773_rep0_Town06_FreeRide_2_None_03_15_14_33_21" 目录

        # 构造 BEV 文件路径（与 JSON 文件名对应）
        bev_file_path = os.path.join(root_dir, "bev", f"{base_name}.png")

        # img_b64_str = encode_image(bev_file_path)


        messages[1]["content"].append(
            {"type": "text", "text": f"This is the {file_i+1}th scenario, {prompt_i}."}
        )
        # messages[1]["content"].append(
        #     {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_b64_str}"}}
        # )
    print(prompt_records)
    save_prompt_record_no_image(prompt_records)
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-11-20",
        messages=messages,
        temperature=0.8,
        response_format=CollisionPredict
    )
    
    # 解析和返回推理结果
    collision_reasoning = completion.choices[0].message.parsed
    return collision_reasoning.to_dict()

In [53]:
def get_collision_reasoning_with_multi_image_no_text(client, record_paths):
    
    # 初始化 messages
    messages = [
        {"role": "system", "content": "You are an expert in collision analysis for autonomous driving scenarios, and your task is to make a 0.5-1 second prediction based on the image and text, whether there will be an accident or not,"
            "you need to consider the behavior of npcs, include lane changes, acceleration and deceleration."},
        {"role": "user", "content": []}  # 确保索引 1 存在
    ]
    
    # 遍历所有图片，分别添加描述
    for file_i in range(len(record_paths)):
        
        # 获取碰撞推理结果
        json_file_name = os.path.basename(record_paths[file_i])  # 获取 '0000.json'
        base_name = os.path.splitext(json_file_name)[0]  # 去掉扩展名，得到 '0000'

        # 获取根目录路径
       root_dir = os.path.dirname(os.path.dirname(record_paths[file_i]))  # 获取到 "RouteScenario_1773_rep0_Town06_FreeRide_2_None_03_15_14_33_21" 目录

        # 构造 BEV 文件路径（与 JSON 文件名对应）
        bev_file_path = os.path.join(root_dir, "bev", f"{base_name}.png")

        img_b64_str = encode_image(bev_file_path)

        messages[1]["content"].append(
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_b64_str}"}}
        )
    print("消息已保存到 messages.json，可手动检查和调整。")
    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=messages,
        temperature=0,
        response_format=CollisionPredict
    )
    
    # 解析和返回推理结果
    collision_reasoning = completion.choices[0].message.parsed
    return collision_reasoning.to_dict()

In [33]:
action_change_frames = ["/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0000.json",
                        "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0000.json",
                        "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0064.json",
                        "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0065.json",
                        ]

In [27]:
def get_next_filenames(file_path, count=4, offset=5):
    """
    生成多个连续的文件名，文件名中的数字部分按照 `offset` 递增。

    :param file_path: 起始文件路径，例如 "0000.png"
    :param count: 需要生成的文件个数
    :param offset: 每次递增的数值
    :return: 返回包含新文件名的列表
    """
    # 获取目录和文件名
    dir_name, file_name = os.path.split(file_path)
    
    # 分离文件名和扩展名
    name, ext = os.path.splitext(file_name)
    
    # 尝试转换文件名中的数字部分
    try:
        start_number = int(name)  # 提取起始数字
        file_list = []
        
        for i in range(count):
            new_number = start_number + (i * offset)  # 计算新的文件编号
            new_file_name = f"{new_number:04d}{ext}"  # 格式化为 4 位数
            file_list.append(os.path.join(dir_name, new_file_name))
        
        return file_list  # 返回包含多个文件名的列表
    except ValueError:
        raise ValueError("文件名不包含有效的数字部分")
    
def extract_number(file_path):
    """提取文件名中的数字部分"""
    file_name = os.path.splitext(os.path.basename(file_path))[0]  # 获取文件名（去除路径和扩展名）
    try:
        return int(file_name)  # 转换为整数
    except ValueError:
        raise ValueError(f"文件名 {file_name} 不包含有效的数字部分")

client = OpenAI(
    base_url="https://yunwu.ai/v1",
    api_key='sk-JXEz0J9T5NwaIHger5rtPjcdzZomXIGHbfuHKm8qpEq3jUKp',
    timeout=120)

all_collision_reasonings = []
for i in range(1, len(action_change_frames)-1):
    frame_start = action_change_frames[i]  # 初始 frame_start
    image_paths = get_next_filenames(frame_start)  # 初始 frame_end
    frame_end = image_paths[-1]
    while extract_number(frame_end) < extract_number(action_change_frames[i + 1]):
        # prompt = f"""Please complete the following tasks step by step:
        #     1️⃣ **Identify Vehicles and Motion Trends from BEV Images:**:
        #     - Detect all vehicles in the bev image and track their positions, orientations.
        #     - Match the same vehicles across multiple frames to ensure consistency in tracking.
        #     - Analyze lane changes, acceleration, deceleration, and sudden behavior shifts (e.g., abrupt lane changes).

        #     2️⃣ **Validate Motion Using Text Descriptions:**:
        #     - Cross-check lane positions, lateral (horizontal) and longitudinal (vertical) distances, and speeds from text with BEV analysis.
        #     - Use textual lane information to verify whether NPCs are changing lanes or maintaining their current trajectory.
        #     - Compare the lateral and longitudinal distance changes between the NPC and the ego vehicle to identify relative action.
        #     - Determine whether the NPC is moving towards the ego vehicle based on lane alignment and distance reduction trends.
        #     - Identify inconsistencies and detect unexpected behaviors (e.g., last-second lane changes).

        #     3️⃣ **Identify High-Risk NPCs**:
        #     - Select NPCs whose motion trends indicate possible trajectory overlap with the ego vehicle.
        #     - Prioritize NPCs that are moving toward the ego vehicle based on lateral and longitudinal distance changes to identify relative action.
        #     - Consider sudden accelerations, lane changes, accelerations and decelerations that may increase the risk of collision.
        #     - Ignore NPCs that maintain a safe distance and consistent motion without signs of potential conflict.
            
        #     4️⃣ **Evaluate Two Types of Collision Risks**:
        #     - **Behavior-induced collisions**: Collisions resulting from actions such as lane changes, deceleration and acceleration.
        #     - **Externally induced collisions**: Unexpected behaviors from npc vehicles that the ego vehicle might not be able to avoid.
        #     - **Refine Collision Judgment with Time and Distance Constraints**:
        #     - Consider that most actions (e.g., lane change, sudden acceleration and deceleration) complete within 0.5-1 second.
        #     - Analyze the distance between the high-risk NPC and the ego vehicle at the start of the action and predict their positions at the completion time.
        #     - If the predicted positions indicate overlap or high-risk proximity, classify it as a likely collision.

        #     5️⃣ **Provide the Final Judgment**:
        #     - Will a collision occur? (Yes/No)
        #     - If a collision is likely, **recommend an avoidance strategy for the autonomous vehicle**.
        #     """
        prompt = f"""Please complete the following tasks step by step:

            1️⃣ **Extract Vehicle Motion Trends from BEV Images**:
            - Detect all vehicles in the BEV image and track their positions and orientations.
            - Match the same vehicles across multiple frames for consistent tracking.
            - Identify sudden behavior shifts (e.g., lane changes, acceleration, deceleration) that may lead to a collision risk.

            2️⃣ **Validate Motion Using Text Descriptions**:
            - Cross-check lane positions, distances, and speeds from text with BEV analysis.
            - Use lane change information to verify NPC trajectory.
            - Compare relative movement between the NPC and ego vehicle to detect potential collision actions.

            3️⃣ **Identify and Evaluate High-Risk NPCs**:
            - Select NPCs with potential trajectory overlap with the ego vehicle.
            - Consider sudden accelerations, decelerations, and lane changes that may increase collision risk.
            - **Predict future positions**: Estimate where the NPC and ego vehicle will be after 0.5-1 seconds.
            - If their predicted paths overlap or get too close, classify as a potential collision.

            4️⃣ **Final Collision Judgment & Avoidance Strategy**:
            - Will a collision occur? (Yes/No)
            - If Yes, recommend an avoidance strategy for the autonomous vehicle.
            """
        collision_reasoning = get_collision_reasoning_with_multi_image(client, prompt, image_paths)

        # 存储文件名和推理结果
        all_collision_reasonings.append({
            "image_paths": image_paths,
            "collision_reasoning": collision_reasoning
        })

        frame_start = frame_end
        # 更新 frame_start 和 frame_end
        image_paths = get_next_filenames(frame_start)  # frame_start +10\
        frame_end = image_paths[-1]  # frame_end 继续 +10

output_file_path = "collision_reasonings_output_end_sequential.json" 
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(all_collision_reasonings, f, ensure_ascii=False, indent=4)

[{'path': 'Final Prompt', 'prompt': 'Please complete the following tasks step by step:\n\n            1️⃣ **Extract Vehicle Motion Trends from BEV Images**:\n            - Detect all vehicles in the BEV image and track their positions and orientations.\n            - Match the same vehicles across multiple frames for consistent tracking.\n            - Identify sudden behavior shifts (e.g., lane changes, acceleration, deceleration) that may lead to a collision risk.\n\n            2️⃣ **Validate Motion Using Text Descriptions**:\n            - Cross-check lane positions, distances, and speeds from text with BEV analysis.\n            - Use lane change information to verify NPC trajectory.\n            - Compare relative movement between the NPC and ego vehicle to detect potential collision actions.\n\n            3️⃣ **Identify and Evaluate High-Risk NPCs**:\n            - Select NPCs with potential trajectory overlap with the ego vehicle.\n            - Consider sudden accelerations, 

[{'path': 'Final Prompt', 'prompt': 'Please complete the following tasks step by step:\n\n            1️⃣ **Extract Vehicle Motion Trends from BEV Images**:\n            - Detect all vehicles in the BEV image and track their positions and orientations.\n            - Match the same vehicles across multiple frames for consistent tracking.\n            - Identify sudden behavior shifts (e.g., lane changes, acceleration, deceleration) that may lead to a collision risk.\n\n            2️⃣ **Validate Motion Using Text Descriptions**:\n            - Cross-check lane positions, distances, and speeds from text with BEV analysis.\n            - Use lane change information to verify NPC trajectory.\n            - Compare relative movement between the NPC and ego vehicle to detect potential collision actions.\n\n            3️⃣ **Identify and Evaluate High-Risk NPCs**:\n            - Select NPCs with potential trajectory overlap with the ego vehicle.\n            - Consider sudden accelerations, 

In [15]:
from openai import OpenAI
prompt = f"""Please complete the following tasks step by step:
    
    1️⃣ **Extract Vehicle Motion Trends from BEV Images**:
    - Detect all vehicles in the BEV image and track their positions and orientations.
    - Match the same vehicles across multiple frames for consistent tracking.
    - Identify sudden behavior shifts (e.g., lane changes, acceleration, deceleration) that may lead to a collision risk.

    2️⃣ **Validate Motion Using Text Descriptions**:
    - Cross-check lane positions, distances, and speeds from text with BEV analysis.
    - Use lane change information to verify NPC trajectory.
    - Compare relative movement between the NPC and ego vehicle to detect potential collision actions.

    3️⃣ **Identify and Evaluate High-Risk NPCs**:
    - Select NPCs with potential trajectory overlap with the ego vehicle.
    - Consider sudden accelerations, decelerations, and lane changes that may increase collision risk.
    - **Predict future positions**: Estimate where the NPC and ego vehicle will be after 0.5-1 seconds.
    - If their predicted paths overlap or get too close, classify as a potential collision.

    4️⃣ **Final Collision Judgment & Avoidance Strategy**:
    - Will a collision occur? (Yes/No)
    - If Yes, recommend an avoidance strategy for the autonomous vehicle.
    """
image_paths=[
    "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0044.json",
    "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0049.json",
    "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0054.json",
    "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0059.json"
]

client = OpenAI(
base_url="https://yunwu.ai/v1",
api_key='sk-JXEz0J9T5NwaIHger5rtPjcdzZomXIGHbfuHKm8qpEq3jUKp',
timeout=120)
collision_reasoning = get_collision_reasoning_with_multi_image(client, prompt, image_paths)

[{'path': 'Final Prompt', 'prompt': 'Please complete the following tasks step by step:\n    \n    1️⃣ **Extract Vehicle Motion Trends from BEV Images**:\n    - Detect all vehicles in the BEV image and track their positions and orientations.\n    - Match the same vehicles across multiple frames for consistent tracking.\n    - Identify sudden behavior shifts (e.g., lane changes, acceleration, deceleration) that may lead to a collision risk.\n\n    2️⃣ **Validate Motion Using Text Descriptions**:\n    - Cross-check lane positions, distances, and speeds from text with BEV analysis.\n    - Use lane change information to verify NPC trajectory.\n    - Compare relative movement between the NPC and ego vehicle to detect potential collision actions.\n\n    3️⃣ **Identify and Evaluate High-Risk NPCs**:\n    - Select NPCs with potential trajectory overlap with the ego vehicle.\n    - Consider sudden accelerations, decelerations, and lane changes that may increase collision risk.\n    - **Predict f

In [16]:
print(collision_reasoning)

### Step 1: Extract Vehicle Motion Trends from BEV Images
#### Observations:
1. **Ego vehicle**:
   - Lane: -5 (remains constant across all frames).
   - Speed: Gradually increasing (4.72 m/s → 5.0 m/s).
   - Acceleration: Fluctuates (1.16 m/s² → 2.27 m/s² → 0.86 m/s²), indicating speed adjustments but consistently moving straight.
2. **NPC1 (left vehicle)**:
   - Lane: Initially in "left 1" (-4), moves to "left 2" (-3) in the second frame and remains there.
   - Lateral Distance from Ego: Increases over frames (2.33 m → 3.43 m → 4.58 m → 4.97 m), indicating NPC1 is moving away from Ego laterally.
   - Longitudinal Distance from Ego: Remains relatively steady (12.76 m → 12.67 m → 12.61 m → 12.71 m), showing parallel forward motion.
   - Speed: Remains stable (approx. 5.2 m/s).
   - Behavior: NPC1 has moved into a new lane (left lane change) and maintains a consistent forward trajectory with no sudden accelerations or decelerations.
3. **NPC2 (right vehicle)**:
   - Lane: Remains in "ri

In [12]:
image_paths= ["/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/bev/0225.png",
            #   "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/bev/0227.png",
            #   "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/bev/0229.png",
            #   "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/bev/0231.png",
              "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/no_collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_17_17_04_10/bev/0235.png"

]
#collision_reasoning = get_collision_reasoning_with_multi_image("请一步一步的分析，你首先需要分析每张图片中npc车辆的行为，然后将所有的图片行为结合预测npc的整体的行为，比如变道，直行，加减速等，然后结合每张图片的文字描述，在各个车辆的速度不会发生显著变化的情况下，预测在最后一张图片结束后1s内碰撞的概率，每个图像间隔0.2s",image_paths)

In [22]:
import os
import glob

def get_selected_json_files(base_dir, n, i, m):
    record_dirs = glob.glob(os.path.join(base_dir, "*/record"))
    selected_files = []
    
    for record_dir in record_dirs:
        json_files = sorted(glob.glob(os.path.join(record_dir, "*.json")))
        
        if len(json_files) < n:
            continue  # 跳过文件不足的情况
        
        start_idx = -n  # 倒数第 n 个文件
        selected_files.extend(json_files[start_idx::-i][:m])  # 从倒数第 n 个开始，每隔 i 个选 m 个
    
    return selected_files

def group_files(files, group_size=4):
    return [list(reversed(files[i:i + group_size])) for i in range(0, len(files), group_size)]

if __name__ == "__main__":
    base_directory = "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp"
    n, i, m = 5, 5, 4  # 示例参数，可调整
    selected_json_files = get_selected_json_files(base_directory, n, i, m)
    
    grouped_files = group_files(selected_json_files, group_size=4)
    
    for group in grouped_files:
        print(group)

['/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0045.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0050.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0055.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_21_17_39_22/record/0060.json']
['/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep23_Town06_FreeRide_4_None_03_21_23_13_20/record/0124.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep23_Tow

In [12]:
import glob
import os
from openai import OpenAI
def get_selected_json_files(base_dir, n, i, m):
    record_dirs = sorted(glob.glob(os.path.join(base_dir, "*/record")))
    selected_files = []
    
    for record_dir in record_dirs:
        json_files = sorted(glob.glob(os.path.join(record_dir, "*.json")))
        
        if len(json_files) < n:
            continue  # 跳过文件不足的情况
        
        start_idx = -n  # 倒数第 n 个文件
        selected_files.extend(json_files[start_idx::-i][:m])  # 从倒数第 n 个开始，每隔 i 个选 m 个
    
    return selected_files

def group_files(files, group_size=4):
    return [list(reversed(files[i:i + group_size])) for i in range(0, len(files), group_size)]

if __name__ == "__main__":
    client = OpenAI(
    base_url="https://yunwu.ai/v1",
    api_key='sk-JXEz0J9T5NwaIHger5rtPjcdzZomXIGHbfuHKm8qpEq3jUKp',
    timeout=120)

    base_directory = "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_uniad_traj/collision"
    n, i, m =  6, 5, 4  # 示例参数，可调整
    selected_json_files = get_selected_json_files(base_directory, n, i, m)
    print(selected_json_files)
    grouped_files = group_files(selected_json_files, group_size=m)
    
    all_collision_reasonings = []
    for group in grouped_files:
#         prompt =(
#     "Please complete the following tasks step by step: "
#     "**1**. Carefully analyze the BEV-view image and identify vehicles that match the given textual descriptions, considering their position, lane, and orientation. "
#     "**2**. Based on the vehicle's heading direction, lane position, and speed, infer its likely behavior trends (e.g., lane change, acceleration, deceleration). "
#     "**3**. Using the last observed state of each vehicle (speed, relative position), along with the predicted behavior trends, determine whether a collision with the ego vehicle is likely within the next 1 second. Provide reasoning for your assessment."
# )
        prompt = f"""Please complete the following tasks step by step:
    
    1️⃣ **Extract Vehicle Motion Trends from BEV Images**:
    - Detect all vehicles in the BEV image and track their positions and orientations.
    - Match the same vehicles across multiple frames for consistent tracking.
    - Identify sudden behavior shifts (e.g., lane changes, acceleration, deceleration) that may lead to a collision risk.

    2️⃣ **Validate Motion Using Text Descriptions**:
    - Cross-check lane positions, distances, and speeds from text with BEV analysis.
    - Use lane change information to verify NPC trajectory.
    - Compare relative movement between the NPC and ego vehicle to detect potential collision actions.

    3️⃣ **Identify and Evaluate High-Risk NPCs**:
    - Select NPCs with potential trajectory overlap with the ego vehicle.
    - Consider sudden accelerations, decelerations, and lane changes that may increase collision risk.
    - **Predict future positions**: Estimate where the NPC and ego vehicle will be after 0.5-1 seconds.
    - If their predicted paths overlap or get too close, classify as a potential collision.

    4️⃣ **Final Collision Judgment & Avoidance Strategy**:
    - Will a collision occur? (Yes/No)
    - If Yes, recommend an avoidance strategy for the autonomous vehicle.
    """

        collision_reasoning = get_collision_reasoning_with_no_image(client, prompt, group)

        # 存储文件名和推理结果
        all_collision_reasonings.append({
            "image_paths": group,
            "collision_reasoning": collision_reasoning
        })

    output_file_path = "collision_reasonings_output_TP_no_images.json" 
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(all_collision_reasonings, f, ensure_ascii=False, indent=4)

['/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_uniad_traj/collision/RouteScenario_1792_rep10_Town12_HazardAtSideLane_1_None_03_28_20_37_36/record/0148.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_uniad_traj/collision/RouteScenario_1792_rep10_Town12_HazardAtSideLane_1_None_03_28_20_37_36/record/0143.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_uniad_traj/collision/RouteScenario_1792_rep10_Town12_HazardAtSideLane_1_None_03_28_20_37_36/record/0138.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_uniad_traj/collision/RouteScenario_1792_rep10_Town12_HazardAtSideLane_1_None_03_28_20_37_36/record/0133.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_uniad_traj/collision/RouteScenario_1792_rep11_Town12_HazardAtSideLane_3_None_03_28_05_45_07/record/0178.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_uniad_traj/collision/RouteScenario_1792_rep11_To

In [13]:
client = OpenAI(
    base_url="https://yunwu.ai/v1",
    api_key='sk-JXEz0J9T5NwaIHger5rtPjcdzZomXIGHbfuHKm8qpEq3jUKp',
    timeout=120)
prompt = (
f"请逐步完成以下任务："
f"1. 综合所有图片，预测 npc 车辆的行为。"
f"2. 根据预测行为，预测 1s 内发生ego和npc碰撞的概率。结合过去1s内ego和npc车辆相对速度和相对距离的变化")
collision_reasoning = get_collision_reasoning_with_multi_image(client, prompt,image_paths)

消息已保存到 messages.json，可手动检查和调整。


In [29]:
prompt = (
    f"请逐步完成以下任务："
    f"1. 每张图片间隔 0.5s，综合所有图片的行为，预测 NPC 车辆的整体趋势（例如：是否变道、加速、减速）。"
    f"2. 根据预测趋势，预测 0.5s 内发生ego和npc碰撞的概率。"
    f"   结合过去 0.5s 内 ego 和 NPC 车辆相对速度和相对距离的变化"
)
image_paths = [
    "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_4_None_03_17_17_04_21/record/0193.json",
    "/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/collision_tcp/RouteScenario_1773_rep0_Town06_FreeRide_4_None_03_17_17_04_21/record/0198.json"
]
collision_reasoning = get_collision_reasoning_with_multi_image(client, prompt, image_paths)


消息已保存到 messages.json，可手动检查和调整。


In [30]:
print(collision_reasoning)

{'final_answer': False, 'reason': "Between the two images, NPC1's speed increased from 4.44 m/s to 4.68 m/s, indicating a slight acceleration. NPC2's speed increased from 3.84 m/s to 4.39 m/s, showing a more significant acceleration. The relative distance between ego and NPC1 decreased slightly, while the distance between ego and NPC2 also decreased. However, the changes in speed and distance do not suggest an immediate risk of collision within the next second, as the relative speeds and distances are not converging rapidly enough to indicate an imminent collision.", 'predict_npc_action': 'NPC1 is accelerating slightly, while NPC2 is accelerating more significantly.'}


In [11]:
all_collision_reasonings = []
output_file_path = "collision_reasonings_output.json"  # 输出文件路径

for file_i in  range(1, len(action_change_frames)):
    prompt = generate_full_prompt(action_change_frames[file_i])
    collision_reasoning = get_collision_reasoning(prompt)
    all_collision_reasonings.append(collision_reasoning.dict())

with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(all_collision_reasonings, f, ensure_ascii=False, indent=4)

NameError: name 'action_change_frames' is not defined

In [ ]:
collision_reasoning = get_collision_reasoning_with_image(final_prompt,"/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/RouteScenario_1773_rep0_Town06_FreeRide_2_None_03_15_14_33_21/bev/0042.png")
print(collision_reasoning)

In [ ]:
for file_i in  range(1, len(action_change_frames)):
    prompt = generate_full_prompt(action_change_frames[file_i])
    collision_reasoning = get_collision_reasoning(prompt)

npc1位于车道-4, 右变道, 初始车道-4, 目标速度4.72 m/s, 当前速度1.96 m/s, 与车道夹角0.00, 横向距离ego -3.75 m, 纵向距离ego 10.00 m, npc1在ego的左边 1车道, 前方。
npc2位于车道-7, 直行, 目标速度5.28 m/s, 当前速度1.96 m/s, 与车道夹角0.00, 横向距离ego 6.75 m, 纵向距离ego -10.00 m, npc2在ego的右边 2车道, 后方。
ego位于车道-5, 当前速度0.17 m/s, 当前加速度0.64 m/s², 正在直行。
npc1位于车道-5, 左变道, 初始车道-5, 目标速度4.44 m/s, 当前速度4.62 m/s, 与车道夹角0.52, 横向距离ego 0.10 m, 纵向距离ego 16.08 m, npc1在ego的右边 0车道, 前方。
npc2位于车道-7, 直行, 目标速度5.28 m/s, 当前速度5.04 m/s, 与车道夹角-0.02, 横向距离ego 7.02 m, 纵向距离ego -1.36 m, npc2在ego的右边 2车道, 后方。
ego位于车道-5, 当前速度4.50 m/s, 当前加速度1.18 m/s², 正在直行。
npc1位于车道-5, 左变道, 初始车道-5, 目标速度4.44 m/s, 当前速度4.33 m/s, 与车道夹角-12.83, 横向距离ego -0.48 m, 纵向距离ego 15.76 m, npc1在ego的左边 0车道, 前方。
npc2位于车道-7, 左变道, 初始车道-7, 目标速度4.44 m/s, 当前速度5.27 m/s, 与车道夹角-0.01, 横向距离ego 7.01 m, 纵向距离ego -1.01 m, npc2在ego的右边 2车道, 后方。
ego位于车道-5, 当前速度4.94 m/s, 当前加速度1.35 m/s², 正在直行。
npc1位于车道-3, 左变道, 初始车道-4, 目标速度5.28 m/s, 当前速度5.00 m/s, 与车道夹角1.54, 横向距离ego -7.05 m, 纵向距离ego 14.22 m, npc1在ego的左边 2车道, 前方。
npc2位于车道-6, 右变道, 初始车道-6, 目标速度4.17 m/s, 当前速度

In [24]:
print(action_change_frames)

['/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/RouteScenario_1773_rep0_Town06_FreeRide_2_None_03_15_14_33_21/record/0000.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/RouteScenario_1773_rep0_Town06_FreeRide_2_None_03_15_14_33_21/record/0001.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/RouteScenario_1773_rep0_Town06_FreeRide_2_None_03_15_14_33_21/record/0042.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/RouteScenario_1773_rep0_Town06_FreeRide_2_None_03_15_14_33_21/record/0048.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/RouteScenario_1773_rep0_Town06_FreeRide_2_None_03_15_14_33_21/record/0104.json', '/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/RouteScenario_1773_rep0_Town06_FreeRide_2_None_03_15_14_33_21/record/0122.json', '/bdata/usrdata/root/monitor_test

In [ ]:
import json

def write_collision_reasoning_to_file(action_change_frames, output_file_path):
    # 创建一个空列表，用于存储结果
    all_collision_reasonings = []

    # 遍历所有 action_change_frames
    for file_i in range(2, 3):
        # 生成 full prompt
        prompt = generate_full_prompt(action_change_frames[file_i])

        # 获取碰撞推理结果
        json_file_name = os.path.basename(action_change_frames[file_i])  # 获取 '0000.json'
        base_name = os.path.splitext(json_file_name)[0]  # 去掉扩展名，得到 '0000'

        # 获取根目录路径
       root_dir = os.path.dirname(os.path.dirname(action_change_frames[file_i]))  # 获取到 "RouteScenario_1773_rep0_Town06_FreeRide_2_None_03_15_14_33_21" 目录

        # 构造 BEV 文件路径（与 JSON 文件名对应）
        bev_file_path = os.path.join(root_dir, "bev", f"{base_name}.png")
        collision_reasoning = get_collision_reasoning_with_image(prompt, bev_file_path)

        # 将 collision_reasoning 转换为字典并添加到结果列表中
        all_collision_reasonings.append(collision_reasoning.dict())

    # 将结果列表写入指定的 JSON 文件
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(all_collision_reasonings, f, ensure_ascii=False, indent=4)

output_file_path = "collision_reasonings_output.json"  # 输出文件路径

# 将结果写入文件
write_collision_reasoning_to_file(action_change_frames, output_file_path)

print(f"Collision reasoning results have been written to {output_file_path}.")


npc1位于车道-4, 右变道, 初始车道-4, 目标速度4.72 m/s, 当前速度1.96 m/s, 与车道夹角0.00, 横向距离ego -3.75 m, 纵向距离ego 10.00 m, npc1在ego的左边 1车道, 前方。
npc2位于车道-7, 直行, 目标速度5.28 m/s, 当前速度1.96 m/s, 与车道夹角0.00, 横向距离ego 6.75 m, 纵向距离ego -10.00 m, npc2在ego的右边 2车道, 后方。
ego位于车道-5, 当前速度0.17 m/s, 当前加速度0.64 m/s², 正在直行。


/tmp/ipykernel_822650/1134941186.py:24: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  all_collision_reasonings.append(collision_reasoning.dict())


npc1位于车道-5, 左变道, 初始车道-5, 目标速度4.44 m/s, 当前速度4.62 m/s, 与车道夹角0.52, 横向距离ego 0.10 m, 纵向距离ego 16.08 m, npc1在ego的右边 0车道, 前方。
npc2位于车道-7, 直行, 目标速度5.28 m/s, 当前速度5.04 m/s, 与车道夹角-0.02, 横向距离ego 7.02 m, 纵向距离ego -1.36 m, npc2在ego的右边 2车道, 后方。
ego位于车道-5, 当前速度4.50 m/s, 当前加速度1.18 m/s², 正在直行。
npc1位于车道-5, 左变道, 初始车道-5, 目标速度4.44 m/s, 当前速度4.33 m/s, 与车道夹角-12.83, 横向距离ego -0.48 m, 纵向距离ego 15.76 m, npc1在ego的左边 0车道, 前方。
npc2位于车道-7, 左变道, 初始车道-7, 目标速度4.44 m/s, 当前速度5.27 m/s, 与车道夹角-0.01, 横向距离ego 7.01 m, 纵向距离ego -1.01 m, npc2在ego的右边 2车道, 后方。
ego位于车道-5, 当前速度4.94 m/s, 当前加速度1.35 m/s², 正在直行。
npc1位于车道-3, 左变道, 初始车道-4, 目标速度5.28 m/s, 当前速度5.00 m/s, 与车道夹角1.54, 横向距离ego -7.05 m, 纵向距离ego 14.22 m, npc1在ego的左边 2车道, 前方。
npc2位于车道-6, 右变道, 初始车道-6, 目标速度4.17 m/s, 当前速度4.23 m/s, 与车道夹角0.10, 横向距离ego 3.52 m, 纵向距离ego -4.26 m, npc2在ego的右边 1车道, 后方。
ego位于车道-5, 当前速度4.90 m/s, 当前加速度0.91 m/s², 正在直行。
npc1位于车道-3, 刹车, 目标速度5.28 m/s, 当前速度5.41 m/s, 与车道夹角-0.04, 横向距离ego -7.11 m, 纵向距离ego 14.98 m, npc1在ego的左边 2车道, 前方。
npc2位于车道-7, 右变道, 初始车道-6, 目标速度4.17 m/s, 当前速度

In [ ]:
#构建场景图


In [5]:
import json


# 提取关系图的函数
def extract_relationship_graph(data):
    relationships = []
    for entity, attributes in data.items():
        if entity != "ego_vehicle":
            speed_diff = abs(attributes["speed"] - data["ego_vehicle"]["speed"])

            if speed_diff <= 0.1:
                relative_speed = "equal"
            elif attributes["speed"] > data["ego_vehicle"]["speed"]:
                relative_speed = "faster"
            else:
                relative_speed = "slower"

            relationships.append({
                "from": entity,
                "to": "ego_vehicle",
                "lane_difference": abs(attributes["current_lane"] - data["ego_vehicle"]["lane"]),
                "relative_position": "right" if attributes["lateral_distance"] > 0 else "left",
                "relative_distance": "front" if attributes["longitudinal_distance"] > 0 else "behind",
                "relative_speed": relative_speed,
                "action": attributes["action"]
            })
    return relationships

data = load_json_file("/bdata/usrdata/root/monitor_test/Bench2Drive/eval_bench2drive220_tcp_only_traj/RouteScenario_1773_rep0_Town06_FreeRide_1_None_03_15_14_33_16/record/0042.json")
relationships = extract_relationship_graph(data)
print(relationships)
# # 生成所有关系图
# relationship_graphs = [extract_relationship_graph(data) for data in json_files]

# # 根据关系图查询对应的 JSON 文件
# def find_json_by_relationship(graphs, json_files, from_entity=None, to_entity=None, min_distance=None, max_distance=None):
#     results = []
#     for i, graph in enumerate(graphs):
#         for relationship in graph:
#             if from_entity and relationship["from"] != from_entity:
#                 continue
#             if to_entity and relationship["to"] != to_entity:
#                 continue
#             if min_distance and relationship["longitudinal_distance"] < min_distance:
#                 continue
#             if max_distance and relationship["longitudinal_distance"] > max_distance:
#                 continue
#             results.append(json_files[i])
#             break
#     return results

# # 示例查询：查找 npc1 到 ego_vehicle 的关系
# matching_json = find_json_by_relationship(relationship_graphs, json_files, from_entity="npc1")

# # 打印结果
# print(json.dumps(matching_json, indent=4))


[{'from': 'npc1', 'to': 'ego_vehicle', 'lane_difference': 1, 'relative_position': 'left', 'relative_distance': 'front', 'relative_speed': 'faster', 'action': 1}, {'from': 'npc2', 'to': 'ego_vehicle', 'lane_difference': 2, 'relative_position': 'right', 'relative_distance': 'behind', 'relative_speed': 'faster', 'action': 2}]
